In [31]:
import requests
import xml.etree.ElementTree as ET

def get_folder_list():
    SAS_TOKEN = "sp=racwdl&st=2023-03-01T13:54:27Z&se=2039-02-28T21:54:27Z&spr=https&sv=2021-06-08&sr=c&sig=eaUtdKd2%2F5W320pmME3B8FiCO4dGrznbBIvGywnIMtE%3D"
    CONTAINER_URL = "https://batstorppnecmesprdclrs03.blob.core.windows.net/superdepotreporting-attachments"

    # Make a request to the container to get the list of blobs
    response = requests.get(CONTAINER_URL + "?restype=container&comp=list&" + SAS_TOKEN)

    folders = set()
    # Parse the XML response to get the list of blob names
    if response.status_code == 200:
        xml = response.text
        root = ET.fromstring(xml)
        for blob in root.findall(".//Blob"):
            blob_name = blob.find("Name").text
            print(f"Blob name found: {blob_name}")  # Debug-Ausgabe
            # Extract the folder name if it contains a '/'
            if '/' in blob_name:
                folder_name = blob_name.split('/')[0]
                print(f"Folder name extracted: {folder_name}")  # Debug-Ausgabe
                folders.add(folder_name)
    else:
        print(f"Failed to retrieve blobs, status code: {response.status_code}")
        print(f"Response text: {response.text}")
    
    return list(folders)

print(get_folder_list())

Blob name found: 0083ad6b-e2fd-4922-9ec3-a38cc7f0d24f
Blob name found: 08f0bd04-12c9-44d4-8519-6b747107aaaf
Blob name found: 09dbbaee-e9fe-4089-8345-97d7942cb89a
Blob name found: 0a94bb22-9897-4230-9902-844197792302
Blob name found: 0e9907c2-7b4e-47c5-acc3-0edad855afa9
Blob name found: 0f510ec2-4a2a-460c-bdb6-068909076ad7
Blob name found: 11a2baf7-f928-48fc-85aa-d3d2b6eb94b6
Blob name found: 13750c58-4498-4a87-9850-bc9a8ae02a4f
Blob name found: 17EB81B2-5D95-4E2D-A6FD-A24657922D88
Blob name found: 19845c60-f788-4bb8-92f4-4d10316b6df9
Blob name found: 19c28c5e-ef00-43bc-95e6-2217b4d9a62c
Blob name found: 1b255a73-688e-4a23-8529-09f275e9d7e9
Blob name found: 1de1ac28-e6f5-4612-9780-5012c47dd4cf
Blob name found: 1e6de203-4bdd-4a43-97b6-4bdac563fcfc
Blob name found: 22537868-0055-4eae-9ceb-f0632472319a
Blob name found: 2406cdb2-914c-4159-81ef-3715d0d6f35b
Blob name found: 24f1366c-37fe-454b-8588-83512efcbd06
Blob name found: 25b8eb25-726c-4b10-89af-2da67e0ecfc3
Blob name found: 265fb971-66

# ERSTELLE ORDNER

In [25]:
import requests

def create_folder(folder_name):
    SAS_TOKEN = "sp=racwdl&st=2023-03-01T13:54:27Z&se=2039-02-28T21:54:27Z&spr=https&sv=2021-06-08&sr=c&sig=eaUtdKd2%2F5W320pmME3B8FiCO4dGrznbBIvGywnIMtE%3D"
    CONTAINER_URL = "https://batstorppnecmesprdclrs03.blob.core.windows.net/superdepotreporting-attachments"
    placeholder_blob_name = folder_name.rstrip('/') + '/placeholder.txt'
    blob_url = f"{CONTAINER_URL}/{placeholder_blob_name}?{SAS_TOKEN}"
    
    headers = {
        'x-ms-blob-type': 'BlockBlob',
        'Content-Length': '0'
    }

    response = requests.put(blob_url, headers=headers)
    
    if response.status_code == 201:
        print(f"Folder '{folder_name}' created successfully.")
    else:
        print(f"Failed to create folder '{folder_name}', status code: {response.status_code}")
        print(f"Response text: {response.text}")



# Datei in Ordner Speichern

In [26]:
import requests
import os

def upload_file_to_folder(file_path, folder_name):
    SAS_TOKEN = "sp=racwdl&st=2023-03-01T13:54:27Z&se=2039-02-28T21:54:27Z&spr=https&sv=2021-06-08&sr=c&sig=eaUtdKd2%2F5W320pmME3B8FiCO4dGrznbBIvGywnIMtE%3D"
    CONTAINER_URL = "https://batstorppnecmesprdclrs03.blob.core.windows.net/superdepotreporting-attachments"
    # Dateiname aus dem Pfad extrahieren
    file_name = os.path.basename(file_path)
    # Vollständigen Blob-Namen erstellen
    blob_name = f"{folder_name.rstrip('/')}/{file_name}"
    blob_url = f"{CONTAINER_URL}/{blob_name}?{SAS_TOKEN}"
    
    headers = {
        'x-ms-blob-type': 'BlockBlob'
    }

    # Datei lesen
    with open(file_path, 'rb') as file_data:
        file_content = file_data.read()
    
    response = requests.put(blob_url, headers=headers, data=file_content)
    
    if response.status_code == 201:
        print(f"File '{file_name}' uploaded successfully to folder '{folder_name}'.")
    else:
        print(f"Failed to upload file '{file_name}' to folder '{folder_name}', status code: {response.status_code}")
        print(f"Response text: {response.text}")

# Beispielaufruf der Funktion



/Library/Python_local/Superdepot Reporting/Data/img/forecast_cogo.png

In [27]:
from PIL import Image

# Pfad zur Bilddatei
image_path = "/Library/Python_local/Superdepot Reporting/Data/img/forecast_cogo.png"

# Bild einlesen
image = Image.open(image_path)

# file im Ordner Hick_Up_Uploads speichern
folder_name = "Hick_Up_Uploads"
upload_file_to_folder(image_path, folder_name)


File 'forecast_cogo.png' uploaded successfully to folder 'Hick_Up_Uploads'.


# ALLE DAITEIN IM ORDNER ANZEIGEN

In [29]:
import requests
import xml.etree.ElementTree as ET
import urllib.parse

def list_blobs_in_folder(container_url, sas_token, folder_name):
    # Anfrage an den Container, um die Liste der Blobs zu erhalten
    container_url_encoded = urllib.parse.quote(container_url, safe='/:')
    sas_token_encoded = urllib.parse.quote(sas_token, safe='=&')
    
    response = requests.get(f"{container_url_encoded}?restype=container&comp=list&{sas_token_encoded}")

    blobs_in_folder = []
    # XML-Antwort parsen, um die Liste der Blob-Namen zu erhalten
    if response.status_code == 200:
        xml = response.text
        root = ET.fromstring(xml)
        for blob in root.findall(".//Blob"):
            blob_name = blob.find("Name").text
            # Überprüfen, ob der Blob-Name mit dem Ordnernamen beginnt
            if blob_name.startswith(folder_name.rstrip('/') + '/'):
                blobs_in_folder.append(blob_name)
    else:
        print(f"Failed to retrieve blobs, status code: {response.status_code}")
        print(f"Response text: {response.text}")

    return blobs_in_folder

# Beispielaufruf der Funktion
SAS_TOKEN = "sp=racwdl&st=2023-03-01T13:54:27Z&se=2039-02-28T21:54:27Z&spr=https&sv=2021-06-08&sr=c&sig=eaUtdKd2%2F5W320pmME3B8FiCO4dGrznbBIvGywnIMtE%3D"
CONTAINER_URL = "https://batstorppnecmesprdclrs03.blob.core.windows.net/your_container_name"
folder_contents = list_blobs_in_folder(CONTAINER_URL, SAS_TOKEN, "Hick_Up_Uploads")
print(folder_contents)

Failed to retrieve blobs, status code: 400
Response text: <?xml version="1.0" encoding="utf-8"?><Error><Code>InvalidResourceName</Code><Message>The specifed resource name contains invalid characters.
RequestId:0bdf757c-b01e-0029-2730-b32f73000000
Time:2024-05-31T07:57:07.6652586Z</Message></Error>
[]
